In [6]:
%reload_ext autoreload
%autoreload 2

from _header import *

---
## Load Models and apply LoRA

In [17]:
# PLM

plm_tokenizer = T5Tokenizer.from_pretrained(
    pretrained_model_name_or_path=BASE_MODEL_PLM,
    do_lower_case=False,
    use_fast=True,
    legacy=False,
)

plm_model, plm_loading_info = T5EncoderModel.from_pretrained(
    pretrained_model_name_or_path=BASE_MODEL_PLM,
    output_loading_info=True,
    # device_map=device,
    # load_in_8bit=False,
    # custom_dropout_rate=0.1,
)

plm_lora_config = LoraConfig(
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q", "k", "v", "o"],
    bias="none",
    modules_to_save=plm_loading_info['missing_keys'],
)

plm_model = peft.get_peft_model(plm_model, plm_lora_config)
plm_model.print_trainable_parameters()

trainable params: 3,932,160 || all params: 1,212,073,984 || trainable%: 0.3244


In [14]:
# LLM
llm_tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=BASE_MODEL_LLM
)

llm_model, llm_loading_info = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_LLM,
    # device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    output_loading_info=True,
)

# llm_lora_config = LoraConfig(
#     inference_mode=False,
#     r=8,
#     lora_alpha=16,
#     lora_dropout=0.05,
#     target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
#     bias="none",
#     modules_to_save=plm_loading_info['missing_keys'],
# )

# llm_model = peft.get_peft_model(llm_model, llm_lora_config)
# llm_model.print_trainable_parameters()

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
llm_model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out

---
## Train

In [ ]:
ds_dummy = {
    "MLEVPVWIPILAFAVGLGLGLLIPHLQKPFQRF": "This protein is involved in membrane transport.",
    "MSLEQKKGADIISKILQIQNSIGKTTSPSTLKT": "This enzyme catalyzes the hydrolysis of ATP.",
    "MKMKQQGLVADLLPNIRVMKTFGHFVFNYYNDN": "This transcription factor regulates gene expression."
}

---
## Inference

In [ ]:
with torch.no_grad():
    protein_features = plm_base_model()['last_hidden_state']
    language_features = llm_base_model()['last_hidden_state']